# WordNet

In [1]:
from collections import defaultdict

import networkx as nx
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import wordnet as wn

## Loading data

### Maslow's paper

In [2]:
maslow_df = pd.read_excel('data/aux/maslow.xlsx')
maslow_df.fillna('', inplace=True)

In [3]:
maslow_df.head()

,PHYSIOLOGICAL,SAFETY,LOVE,ESTEEM,SELF_ACTUALIZATION,SELF_TRANSCENDENCE
0,food,freedom,love,esteem,write,volunteering
1,water,philosophy,community,evaluation,poetry,help
2,salt,safety,parents,respect,history,morality
3,sugar,stable,family,capacity,philosophy,ethics
4,protein,sunniness,sex,achievement,culture,volunteering


### Other sources

In [4]:
others_dict = defaultdict(list)
with open('data/aux/maslow-others.txt') as f:
    c = 1
    for line in f:
        if c % 7 != 1 and c % 7 != 0:
            col, vals = line.split(':')
            vals = vals.strip()
            if vals:
                col = col.replace('-', '_').upper()
                vals = vals.split(', ')
                others_dict[col].extend(vals)
        c += 1

others_df = pd.DataFrame(
    dict([(k, pd.Series(v)) for k, v in others_dict.items()]),
    columns=['PHYSIOLOGICAL', 'SAFETY', 'LOVE', 'ESTEEM', 'SELF_ACTUALIZATION', 'SELF_TRANSCENDENCE']
)
others_df.fillna('', inplace=True)

In [5]:
others_df.head()

,PHYSIOLOGICAL,SAFETY,LOVE,ESTEEM,SELF_ACTUALIZATION,SELF_TRANSCENDENCE
0,air,assault,friends,wise,capable,
1,water,murder,family,decision-maker,education,
2,calories,homicide,community,status,university,
3,nutrients,chaos,divorce,confidence,college,
4,food,war,child,competent,creativity,


### Concatenating data

In [6]:
needs_df = pd.concat([maslow_df, others_df], ignore_index=True)

In [7]:
needs_df.head()

,PHYSIOLOGICAL,SAFETY,LOVE,ESTEEM,SELF_ACTUALIZATION,SELF_TRANSCENDENCE
0,food,freedom,love,esteem,write,volunteering
1,water,philosophy,community,evaluation,poetry,help
2,salt,safety,parents,respect,history,morality
3,sugar,stable,family,capacity,philosophy,ethics
4,protein,sunniness,sex,achievement,culture,volunteering


## Processing data

In [8]:
def all_hypernyms(ss):
    return ss.closure(lambda x: x.hypernyms())

In [9]:
def all_hyponyms(ss):
    return ss.closure(lambda x: x.hyponyms())

In [10]:
for need in needs_df:
    print(need, needs_df[need].nunique())

PHYSIOLOGICAL 53
SAFETY 142
LOVE 71
ESTEEM 70
SELF_ACTUALIZATION 124
SELF_TRANSCENDENCE 6


### Physiological needs

In [11]:
physiological_raw_needs = needs_df.PHYSIOLOGICAL.sort_values().unique()

In [12]:
physiological_raw_needs

array(['', 'air', 'appetite', 'bread', 'breath', 'breathing', 'calcium',
       'calories', 'cigarette', 'clothing', 'comfort', 'defecate', 'drink',
       'eat', 'exercise', 'fat', 'fatigue', 'feed', 'food', 'health',
       'heat', 'hormones', 'hunger', 'hungry', 'hydrogen', 'movement',
       'nourishment', 'nutrients', 'oxygen', 'pleasure', 'protein', 'rest',
       'safe', 'salt', 'secure', 'sex', 'shelter', 'sleep', 'sleepiness',
       'smell', 'smoke', 'stroking', 'sugar', 'taste', 'temperature',
       'thirst', 'thirsty', 'tickling', 'urinate', 'vitamin', 'vitamins',
       'warmth', 'water'], dtype=object)

In [13]:
physiological_needs_removed = [
    '', 'bread', 'calcium', 'hormones', 'hydrogen', 'movement', 'salt', 'sugar', 'temperature', 'vitamins',
]

In [14]:
physiological_needs = sorted(set(physiological_raw_needs) - set(physiological_needs_removed))

In [15]:
print(physiological_needs)

['air', 'appetite', 'breath', 'breathing', 'calories', 'cigarette', 'clothing', 'comfort', 'defecate', 'drink', 'eat', 'exercise', 'fat', 'fatigue', 'feed', 'food', 'health', 'heat', 'hunger', 'hungry', 'nourishment', 'nutrients', 'oxygen', 'pleasure', 'protein', 'rest', 'safe', 'secure', 'sex', 'shelter', 'sleep', 'sleepiness', 'smell', 'smoke', 'stroking', 'taste', 'thirst', 'thirsty', 'tickling', 'urinate', 'vitamin', 'warmth', 'water']


In [16]:
physiological_all_synsets_list = [(need, ss) for need in physiological_needs for ss in wn.synsets(need)]

In [17]:
len(physiological_all_synsets_list)

262

In [18]:
for need, ss in physiological_all_synsets_list:
    print('*', need, '-', ss.name(), '-', ss.definition())

* air - air.n.01 - a mixture of gases (especially oxygen) required for breathing; the stuff that the wind consists of
* air - air.n.02 - the region above the ground
* air - air.n.03 - a distinctive but intangible quality surrounding a person or thing
* air - breeze.n.01 - a slight wind (usually refreshing)
* air - atmosphere.n.03 - the mass of air surrounding the Earth
* air - air.n.06 - once thought to be one of four elements composing the universe (Empedocles)
* air - tune.n.01 - a succession of notes forming a distinctive sequence
* air - air.n.08 - medium for radio and television broadcasting
* air - air_travel.n.01 - travel via aircraft
* air - air_out.v.01 - expose to fresh air
* air - air.v.02 - be broadcast
* air - air.v.03 - broadcast over the airwaves, as in radio or television
* air - publicize.v.01 - make public
* air - air.v.05 - expose to warm or heated air, so as to dry
* air - vent.v.02 - expose to cool or cold air so as to cool or freshen
* appetite - appetite.n.01 - a

* taste - taste.n.03 - delicate discrimination (especially of aesthetic values)
* taste - taste.n.04 - a brief experience of something
* taste - taste.n.05 - a small amount eaten or drunk
* taste - taste.n.06 - the faculty of distinguishing sweet, sour, bitter, and salty properties in the mouth
* taste - taste.n.07 - a kind of sensing; distinguishing substances by means of the taste buds
* taste - taste.v.01 - have flavor; taste of something
* taste - taste.v.02 - perceive by the sense of taste
* taste - sample.v.01 - take a sample of
* taste - smack.v.03 - have a distinctive or characteristic taste
* taste - taste.v.05 - distinguish flavors
* taste - taste.v.06 - experience briefly
* thirst - thirst.n.01 - a physiological need to drink
* thirst - hunger.n.02 - strong desire for something (not food or drink)
* thirst - thirst.v.01 - feel the need to drink
* thirst - crave.v.01 - have a craving, appetite, or great desire for
* thirsty - thirsty.s.01 - needing moisture
* thirsty - thirst

In [19]:
physiological_synsets_list = [
    'air.n.01', # air
    'appetite.n.01', # appetite
    'bread.n.01', # bread
    'breath.n.01', 'breath.n.02', 'breath.n.03', # breath
    'breathing.n.01', 'breathe.v.01', 'rest.v.02', # breathing
    'calorie.n.01', 'calorie.n.02', # calories
    'cigarette.n.01', # cigarette
    'clothing.n.01', 'dress.v.02', 'clothe.v.03', # clothing
    'comfort.n.01', 'comfort.n.05',  # comfort
    'stool.v.04', # defecate
    'drink.n.01', 'drink.n.02', 'beverage.n.01', 'drink.v.01', 'drink.v.02', 'drink.v.05', # drink
    'eat.v.01', 'eat.v.02', # eat
    'exercise.n.01', 'exercise.v.03', 'exercise.v.04', # exercise
    'fat.n.01', 'fatness.n.01', 'fat.a.01', 'fatty.a.01', # fat
    'fatigue.n.01', 'fatigue.n.03', 'tire.v.01', 'tire.v.02', # fatigue
    'feed.n.01', 'feed.v.01', 'feed.v.02', 'feed.v.03', 'feed.v.07', # feed
    'food.n.01', 'food.n.02', # food
    'health.n.01', 'health.n.02', # health
    'heat.n.01', 'hotness.n.01', 'heat.n.03', 'heat.v.01', 'heat.v.02', 'inflame.v.04', 'heat.v.04', # heat
    'hunger.n.01', 'hunger.v.01', 'starve.v.01', # hunger
    'hungry.a.01', # hungry
    'nutriment.n.01', 'nourishment.n.02', # nourishment
    'food.n.01', # nutrients
    'oxygen.n.01', # oxygen
    'pleasure.n.01', 'joy.n.02', 'pleasure.n.04', 'pleasure.n.05', # pleasure
    'protein.n.01', # protein
    'rest.n.02', 'respite.n.04', 'rest.n.04', 'rest.v.01', 'rest.v.02', 'rest.v.03', 'rest.v.05', # rest
    'condom.n.01', 'safe.a.01', 'safe.s.02', #safe
    'secure.a.02', # secure
    'sexual_activity.n.01', 'sex.n.03', 'sex.n.04', 'arouse.v.07', # sex
    'shelter.n.01', 'shelter.n.02', 'protection.n.04', 'shelter.n.05', 'shelter.v.01', # shelter
    'sleep.n.01', 'sleep.n.02', 'sleep.n.03', 'sleep.v.01', 'sleep.v.02', # sleep
    'sleepiness.n.01', # sleepiness
    'smell.n.01', 'olfactory_property.n.01', 'smell.n.04', 'smell.n.05', 'smell.v.01', # smell
    'roll_of_tobacco.n.01', 'pot.n.09', 'smoke.n.07', 'smoke.v.01', # smoke
    'stroke.n.05', 'stroke.v.01', # stroking
    'taste.n.01', 'preference.n.01', 'taste.n.05', 'taste.v.02', 'taste.v.05', # taste
    'thirst.n.01', 'thirst.v.01', # thirst
    'thirsty.s.01', 'thirsty.a.02', # thirsty
    'tickle.n.02', 'tickle.v.01', 'thrill.v.02', 'tickle.v.03', 'tickling.s.01', # tickling
    'urinate.v.01', 'make.v.49', # urinate
    'vitamin.n.01', # vitamin
    'heat.n.03', 'warmheartedness.n.01', 'warmth.n.03', # warmth
    'water.n.01', 'urine.n.01', 'water.n.06', # water
]

In [146]:
len(physiological_synsets_list)

126

In [172]:
aux = physiological_synsets_list.copy()

diff = 1
while(diff):
    for ss_str in aux.copy():
        ss1 = wn.synset(ss_str)
        for lemma in ss1.lemmas():
            for antonym in lemma.antonyms():
                ss2 = antonym.synset()
                aux.append(ss2.name())
    aux = list(set(aux))
    v1 = len(aux)
    
    for ss_str in aux.copy():
        ss1 = wn.synset(ss_str)
        aux.append(ss1.name())
        aux.extend(ss2.name() for ss2 in all_hyponyms(ss1))
    aux = list(set(aux))
    v2 = len(aux)
    
    diff = v2 - v1

physiological_full_synsets_list = aux

In [173]:
len(physiological_full_synsets_list)

4709

### Safety needs

In [38]:
safety_raw_needs = needs_df.SAFETY.sort_values().unique()

In [39]:
safety_raw_needs

array(['', 'abuse', 'accident', 'accommodation', 'anxiety', 'anxious',
       'arrange', 'assault', 'attack', 'attainment', 'catastrophe',
       'ceremonial', 'chaos', 'climate', 'coherent', 'comfort',
       'community', 'consistency', 'contingency', 'crime', 'criminal',
       'crisis', 'danger', 'dangerous', 'darkness', 'death', 'defend',
       'defense', 'disaster', 'discipline', 'disciplined', 'disease',
       'disorganization', 'divorce', 'economy', 'emergency', 'employment',
       'endangered', 'equilibrium', 'fairness', 'familiar', 'family',
       'fear', 'financial', 'formulas', 'freedom', 'grievance', 'harm',
       'harshly', 'health', 'homicide', 'hostile', 'housing', 'illness',
       'inconsistency', 'injury', 'injustice', 'insurance', 'job',
       'justice', 'known', 'law', 'leader', 'meaningful', 'mug', 'murder',
       'neat', 'nightmares', 'order', 'orderly', 'organization',
       'organize', 'organized', 'outline', 'overwhelming', 'pain', 'panic',
       'pare

In [40]:
safety_needs_removed = [
    '', 'attainment', 'climate', 'community', 'divorce', 'family', 'formulas', 'leader', 'neat', 'parents',
    'philosophy', 'religion', 'science', 'separation', 'threats',
]

In [41]:
safety_needs = sorted(set(safety_raw_needs) - set(safety_needs_removed))

In [42]:
print(safety_needs)

['abuse', 'accident', 'accommodation', 'anxiety', 'anxious', 'arrange', 'assault', 'attack', 'catastrophe', 'ceremonial', 'chaos', 'coherent', 'comfort', 'consistency', 'contingency', 'crime', 'criminal', 'crisis', 'danger', 'dangerous', 'darkness', 'death', 'defend', 'defense', 'disaster', 'discipline', 'disciplined', 'disease', 'disorganization', 'economy', 'emergency', 'employment', 'endangered', 'equilibrium', 'fairness', 'familiar', 'fear', 'financial', 'freedom', 'grievance', 'harm', 'harshly', 'health', 'homicide', 'hostile', 'housing', 'illness', 'inconsistency', 'injury', 'injustice', 'insurance', 'job', 'justice', 'known', 'law', 'meaningful', 'mug', 'murder', 'nightmares', 'order', 'orderly', 'organization', 'organize', 'organized', 'outline', 'overwhelming', 'pain', 'panic', 'peace', 'peaceful', 'predictable', 'problems', 'property', 'protect', 'protection', 'protector', 'punishment', 'quarreling', 'rage', 'reassurance', 'reliable', 'rhythm', 'risk', 'routine', 'rules', 'sa

In [43]:
safety_all_synsets_list = [(need, ss) for need in safety_needs for ss in wn.synsets(need)]

In [44]:
len(safety_all_synsets_list)

609

In [45]:
for need, ss in safety_all_synsets_list:
    print('*', need, '-', ss.name(), '-', ss.definition())

* abuse - maltreatment.n.01 - cruel or inhumane treatment
* abuse - abuse.n.02 - a rude expression intended to offend or hurt
* abuse - misuse.n.01 - improper or excessive use
* abuse - mistreat.v.01 - treat badly
* abuse - pervert.v.03 - change the inherent purpose or function of something
* abuse - abuse.v.03 - use foul or abusive language towards
* abuse - abuse.v.04 - use wrongly or improperly or excessively
* accident - accident.n.01 - an unfortunate mishap; especially one causing damage or injury
* accident - accident.n.02 - anything that happens suddenly or by chance without an apparent cause
* accommodation - adjustment.n.01 - making or becoming suitable; adjusting to circumstances
* accommodation - accommodation.n.02 - a settlement of differences
* accommodation - accommodation.n.03 - in the theories of Jean Piaget: the modification of internal representations in order to accommodate a changing knowledge of reality
* accommodation - accommodation.n.04 - living quarters provide

* law - law.n.01 - the collection of rules imposed by authority
* law - law.n.02 - legal document setting forth rules governing a particular kind of activity
* law - law.n.03 - a rule or body of rules of conduct inherent in human nature and essential to or binding upon human society
* law - law.n.04 - a generalization that describes recurring facts or events in nature
* law - jurisprudence.n.01 - the branch of philosophy concerned with the law and the principles that lead courts to make the decisions they do
* law - law.n.06 - the learned profession that is mastered by graduate study in a law school and that is responsible for the judicial system
* law - police.n.01 - the force of policemen and officers
* meaningful - meaningful.a.01 - having a meaning or purpose
* mug - mug.n.01 - the quantity that can be held in a mug
* mug - chump.n.01 - a person who is gullible and easy to take advantage of
* mug - countenance.n.03 - the human face (`kisser' and `smiler' and `mug' are informal term

In [17]:
safety_synsets_list = [
    'maltreatment.n.01', 'abuse.n.02', 'mistreat.v.01', 'abuse.v.03', # abuse
    'accident.n.01', 'accident.n.02', # accident
    'accommodation.n.05', # accommodation
    'anxiety.n.01', # anxiety
]

In [146]:
len(physiological_synsets_list)

126

In [172]:
aux = physiological_synsets_list.copy()

diff = 1
while(diff):
    for ss_str in aux.copy():
        ss1 = wn.synset(ss_str)
        for lemma in ss1.lemmas():
            for antonym in lemma.antonyms():
                ss2 = antonym.synset()
                aux.append(ss2.name())
    aux = list(set(aux))
    v1 = len(aux)
    
    for ss_str in aux.copy():
        ss1 = wn.synset(ss_str)
        aux.append(ss1.name())
        aux.extend(ss2.name() for ss2 in all_hyponyms(ss1))
    aux = list(set(aux))
    v2 = len(aux)
    
    diff = v2 - v1

physiological_full_synsets_list = aux

In [173]:
len(physiological_full_synsets_list)

4709